In [2]:
import pandas as pd

In [3]:
father_data = pd.read_csv("Father Genome.csv")
mother_data = pd.read_csv("Mother Genome.csv")
child1_data = pd.read_csv("Child 1 Genome.csv")

C:\Users\Hp\AppData\Local\Temp\ipykernel_7404\757149404.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  father_data = pd.read_csv("Father Genome.csv")
C:\Users\Hp\AppData\Local\Temp\ipykernel_7404\757149404.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  mother_data = pd.read_csv("Mother Genome.csv")
C:\Users\Hp\AppData\Local\Temp\ipykernel_7404\757149404.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  child1_data = pd.read_csv("Child 1 Genome.csv")


In [4]:
father_data.head()

,# rsid,chromosome,position,genotype
0,rs12564807,1,734462,AA
1,rs3131972,1,752721,AG
2,rs148828841,1,760998,AC
3,rs12124819,1,776546,AA
4,rs115093905,1,787173,GG


In [6]:
merged_data = father_data.merge(mother_data, on=["# rsid", "chromosome", "position"], suffixes=('_father', '_mother'))
merged_data = merged_data.merge(child1_data, on=["# rsid", "chromosome", "position"])
merged_data.rename(columns={"genotype": "genotype_child"}, inplace=True)

In [7]:
def analyze_inheritance(row):
    father_alleles = set(row['genotype_father'])
    mother_alleles = set(row['genotype_mother'])
    child_alleles = set(row['genotype_child'])
    
    # Check if child's alleles are consistent with parental alleles
    if child_alleles.issubset(father_alleles.union(mother_alleles)):
        return "Consistent"
    else:
        return "Inconsistent"

In [8]:
merged_data["inheritance"] = merged_data.apply(analyze_inheritance, axis=1)

In [11]:
print(merged_data)

             # rsid chromosome  position genotype_father genotype_mother  \
0        rs12564807          1    734462              AA              AA   
1         rs3131972          1    752721              AG              GG   
2       rs148828841          1    760998              AC              CC   
3        rs12124819          1    776546              AA              AG   
4       rs115093905          1    787173              GG              GG   
...             ...        ...       ...             ...             ...   
601797     i4000757         MT     16526               G               G   
601798      i701671         MT     16526               G               G   
601799     i4990307         MT     16527               C               C   
601800     i4000756         MT     16540               C               C   
601801     i3001931         MT     16547               C               C   

       genotype_child inheritance  
0                  AA  Consistent  
1              

In [12]:
def check_homozygosity(genotype):
    # Check if the genotype is empty or malformed
    if not genotype or len(genotype) < 2:  # Empty or too short
        return "Malformed or Empty"
    # Check if both alleles are the same
    return "Homozygous" if genotype[0] == genotype[1] else "Heterozygous"

In [14]:
merged_data["child_homozygosity"] = merged_data["genotype_child"].apply(check_homozygosity)
print(merged_data[["# rsid", "chromosome", "position", "genotype_father", 
                   "genotype_mother", "genotype_child", "inheritance", 
                   "child_homozygosity"]])

             # rsid chromosome  position genotype_father genotype_mother  \
0        rs12564807          1    734462              AA              AA   
1         rs3131972          1    752721              AG              GG   
2       rs148828841          1    760998              AC              CC   
3        rs12124819          1    776546              AA              AG   
4       rs115093905          1    787173              GG              GG   
...             ...        ...       ...             ...             ...   
601797     i4000757         MT     16526               G               G   
601798      i701671         MT     16526               G               G   
601799     i4990307         MT     16527               C               C   
601800     i4000756         MT     16540               C               C   
601801     i3001931         MT     16547               C               C   

       genotype_child inheritance  child_homozygosity  
0                  AA  Consiste

In [15]:
merged_data.to_csv("Inheritance_Analysis.csv", index=False)

In [16]:
trait_database = {
    "rs815743": {"trait": "Eye Color Variation", "risk_allele": "A"},
    "rs823552": {"trait": "Increased Risk of Diabetes", "risk_allele": "G"},
    "rs780331": {"trait": "Height Variation", "risk_allele": "T"},
    "rs709746": {"trait": "Lactose Intolerance", "risk_allele": "C"},
    "rs976199": {"trait": "Cardiovascular Risk", "risk_allele": "G"},
    "rs123456": {"trait": "Skin Pigmentation", "risk_allele": "C"},
    "rs654321": {"trait": "Obesity Risk", "risk_allele": "T"},
    "rs789012": {"trait": "Allergy Susceptibility", "risk_allele": "A"},
    "rs345678": {"trait": "Bone Density Variation", "risk_allele": "G"},
    "rs901234": {"trait": "Immune Response", "risk_allele": "T"}
}

In [22]:
def predict_traits(row):
    rsid = row["# rsid"]
    if rsid in trait_database:
        risk_allele = trait_database[rsid]["risk_allele"]
        trait = trait_database[rsid]["trait"]
        if risk_allele in row["genotype_child"]:
            return f"Risk for {trait}"
        else:
            return f"No risk for {trait}"
    return "No data"

In [23]:
merged_data["trait_prediction"] = merged_data.apply(predict_traits, axis=1)

In [24]:
print(merged_data[["# rsid", "genotype_child", "trait_prediction"]])

             # rsid genotype_child trait_prediction
0        rs12564807             AA          No data
1         rs3131972             AG          No data
2       rs148828841             AC          No data
3        rs12124819             AG          No data
4       rs115093905             GG          No data
...             ...            ...              ...
601797     i4000757              G          No data
601798      i701671              G          No data
601799     i4990307              C          No data
601800     i4000756              C          No data
601801     i3001931              C          No data

[601802 rows x 3 columns]


In [25]:
merged_data.to_csv("Child_result_Trait_Predictions.csv", index=False)

In [26]:
data = pd.read_csv("Child_result_Trait_Predictions.csv")

C:\Users\Hp\AppData\Local\Temp\ipykernel_7404\3612181664.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("Child_result_Trait_Predictions.csv")


In [27]:
def calculate_confidence(data):
    # Count total SNPs
    total_snps = len(data)
    # Count consistent SNPs
    consistent_snps = len(data[data['inheritance'] == 'Consistent'])
    # Calculate confidence percentage
    confidence = (consistent_snps / total_snps) * 100
    return total_snps, consistent_snps, confidence

In [28]:
total_snps, consistent_snps, confidence = calculate_confidence(data)

# Display results
print("Analysis of Child_result_Trait_Predictions.csv:")
print(f"Total SNPs analyzed: {total_snps}")
print(f"Consistent SNPs: {consistent_snps}")
print(f"Confidence that the child belongs to these parents: {confidence:.2f}%")

# Save the results to a new CSV file with additional summary information
data['confidence'] = f"{confidence:.2f}%"
data.to_csv("Child_Inheritance_Confidence_Analysis.csv", index=False)

Analysis of Child_result_Trait_Predictions.csv:
Total SNPs analyzed: 601802
Consistent SNPs: 596351
Confidence that the child belongs to these parents: 99.09%
